목적 : 부실여부 판단하기
==============
- 조건 1 : 이자보생배율 3년 연속 1 미만
- 조건 2 : 영업현금흐름 3년 연속 (-)
- 조건 3 : 감사의견 거절 혹은 부적절

import module

In [1]:
import pandas as pd

pd.options.display.float_format = "{:.2f}".format

read csv file

In [2]:
kskqkn = pd.read_csv("../datasets/상장_이자보상배율.csv", index_col=0)

In [3]:
kskqkn = kskqkn[
    [
        "회사명",
        "거래소코드",
        "회계년도",
        "이자보상배율(이자비용)",
        "[U01250010]   이자비용(천원)",
        "[U01230000]영업이익(손실)(천원)",
    ]
]

회계 데이터 전처리

In [4]:
# 회계년도의 회계월이 12월인 데이터만 남김
kskqkn["회계년도"] = pd.to_datetime(kskqkn["회계년도"])
kskqkn = kskqkn[kskqkn["회계년도"].dt.month == 12].reset_index(drop=True)
kskqkn["회계년도"] = kskqkn["회계년도"].dt.year

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\2328377546.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  kskqkn["회계년도"] = pd.to_datetime(kskqkn["회계년도"])


In [5]:
# 이상치에 0 대입
kskqkn.loc[
    (kskqkn["이자보상배율(이자비용)"] == 999999999) & (kskqkn["[U01230000]영업이익(손실)(천원)"] < 0),
    "이자보상배율(이자비용)",
] = 0

In [6]:
kskqkn.head(3)

,회사명,거래소코드,회계년도,이자보상배율(이자비용),[U01250010] 이자비용(천원),[U01230000]영업이익(손실)(천원)
0,(주)CMG제약,58820,2012,-7.45,542880.00,-4045140.00
1,(주)CMG제약,58820,2013,0.73,1133753.00,822547.00
2,(주)CMG제약,58820,2014,3.63,122840.00,446018.00


read csv file

In [7]:
df = pd.read_csv("../datasets/상장_부실판단.csv", encoding='cp949')
df.head(3)

,회사명,거래소코드,회계년도,감사의견코드,[U01410000]영업활동으로 인한 현금흐름(*)(천원),이자보상배율(이자비용)
0,(주)CMG제약,58820,Dec-12,UQ,-1453276.00,-7.45
1,(주)CMG제약,58820,Dec-13,UQ,-3891420.00,0.73
2,(주)CMG제약,58820,Dec-14,UQ,730447.00,3.63


회계 데이터 전처리

In [8]:
# 회계년도의 회계월이 12월인 데이터만 남김
df["회계년도"] = pd.to_datetime(df["회계년도"], format="%b-%y")
df = df[df["회계년도"].dt.month == 12].reset_index(drop=True)
df["회계년도"] = df["회계년도"].dt.year

In [9]:
# 데이터 합치기
listed_merged_df = pd.merge(
    df, kskqkn[["거래소코드", "회계년도", "이자보상배율(이자비용)"]], on=["거래소코드", "회계년도"], how="left"
)

In [10]:
listed_merged_df.columns

Index(['회사명', '거래소코드', '회계년도', '감사의견코드', '[U01410000]영업활동으로 인한 현금흐름(*)(천원)',
       '이자보상배율(이자비용)_x', '이자보상배율(이자비용)_y'],
      dtype='object')

In [11]:
listed_df = listed_merged_df[
    [
        "회사명",
        "거래소코드",
        "회계년도",
        "감사의견코드",
        "[U01410000]영업활동으로 인한 현금흐름(*)(천원)",
        "이자보상배율(이자비용)_y",
    ]
]

In [12]:
listed_df.rename(columns={"이자보상배율(이자비용)_y": "이자보상배율(이자비용)"}, inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\1554775916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listed_df.rename(columns={"이자보상배율(이자비용)_y": "이자보상배율(이자비용)"}, inplace=True)


# 부실 판단

In [13]:
concat_df = pd.DataFrame()

##### 감사의견코드 부실 정의

In [14]:
df = listed_df.copy()

In [15]:
# 초기값은 None
df["감사부실"] = None

In [16]:
code_df = pd.DataFrame()

In [17]:
# 감사의견 부실판정
for i in range(len(df)):
    if df["감사의견코드"][i] in ["AG", "DI", "DS", "DU"]:
        df["감사부실"][i] = 1
    elif pd.isna(df.loc[i, "감사의견코드"]):
        df["감사부실"][i] = None
    else:
        df["감사부실"][i] = 0

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\3998903815.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감사부실"][i] = 0
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\3998903815.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감사부실"][i] = 0
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\3998903815.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감사부실"][i] = 0
C:\Users\jay_hook\AppData\Local\Temp\ipy

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\3998903815.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감사부실"][i] = 0
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\3998903815.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감사부실"][i] = 0
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\3998903815.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감사부실"][i] = 0
C:\Users\jay_hook\AppData\Local\Temp\ipy

In [18]:
# 사용할 감사부실 회사
bank1 = df.loc[df["감사부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
bank1_com = bank1[["거래소코드", "회계년도"]]
bank1_com

,거래소코드,회계년도
72,35290,2019
343,106520,2022
423,145210,2018
657,141020,2021
666,65150,2019
926,197210,2019
975,127160,2020
1099,38340,2020
1242,66410,2022
1263,299910,2021


In [19]:
# 3년 연속 회계 데이터 있는 회사
nan_list = []

for code, year in zip(bank1_com["거래소코드"], bank1_com["회계년도"]):
    test = df[df["거래소코드"] == code]
    year_index = test[test["회계년도"] == year].index
    if (code != df["거래소코드"].iloc[year_index[0] - 2]) | (year - 2) != df["회계년도"].iloc[
        year_index[0] - 2
    ]:
        nan_list.append(code)

print(nan_list)

[950160]


In [20]:
# df[df["거래소코드"] == year_nan_list]
for code in nan_list:
    df = df[df["거래소코드"] != code]

In [21]:
grouped = df.groupby("거래소코드")

In [22]:
year_nan_list = []

for code, group in grouped:
    # print(code, group)
    # print(len(group))
    len_group = len(group)
    if len_group < 3:
        year_nan_list.append(code)
print(year_nan_list)

[18700, 58970, 59270, 67370, 86710, 99390, 99430, 102370, 107600, 112290, 114840, 129920, 136410, 137080, 137310, 146320, 159010, 163730, 168360, 169670, 176590, 187660, 189330, 199800, 200350, 203400, 204610, 236810, 239890, 247660, 248020, 251120, 254160, 257720, 260970, 261780, 262840, 270660, 273640, 276040, 276730, 277810, 288980, 289220, 290090, 291810, 294570, 296640, 303530, 308080, 310210, 311320, 311960, 314140, 314930, 315640, 318660, 321820, 322310, 328130, 330350, 333620, 334970, 338220, 340930, 341170, 346010, 347700, 348030, 348340, 348370, 351330, 352480, 352700, 352910, 353590, 354390, 356890, 357230, 357580, 357880, 360070, 361390, 361570, 361670, 362320, 363250, 363260, 365270, 365340, 365900, 366030, 367000, 368600, 368770, 368970, 371950, 372170, 372800, 373200, 376180, 376930, 376980, 377030, 377220, 377330, 377450, 377480, 377740, 378800, 379390, 380540, 382480, 382800, 382840, 382900, 383310, 383930, 388050, 388610, 388720, 389030, 389140, 389260, 389470, 390110

In [23]:
# df[df["거래소코드"] == year_nan_list]
for code in year_nan_list:
    df = df[df["거래소코드"] != code]

In [24]:
df = df.reset_index(drop=True)

In [25]:
# # 사용할 감사부실 회사
insolvent_com = df.loc[df["감사부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
insolvent_codes = insolvent_com["거래소코드"].unique()

non_insolvent_com = df[~df["거래소코드"].isin(insolvent_codes)]  # 부실선정 기업 제거
non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

non_insolvent_com.isna().sum()

print(len(insolvent_codes))
print(len(non_insolvent_com) - len(non_insolvent_com.isna().sum()))
print(len(non_insolvent_com.isna().sum()))
print(len(df["거래소코드"].unique()))

55
925
7
987


C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\3116497719.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [26]:
code_df = pd.concat([insolvent_com, non_insolvent_com])

##### 이자보상배율 부실정의

In [27]:
df = listed_df.copy()

In [28]:
ratio_df = pd.DataFrame()

In [29]:
# 초기값은 None
df["이보배부실"] = None

In [30]:
grouped = df.groupby("거래소코드")

In [31]:
# 이자보상배율 부실판정
for code, group in grouped:
    # print(code, group)
    group_len = len(group)
    # print(group_len)
    for i in range(group_len - 2):
        cond1 = group.iloc[i]["이자보상배율(이자비용)"] < 1
        cond2 = group.iloc[i + 1]["이자보상배율(이자비용)"] < 1
        cond3 = group.iloc[i + 2]["이자보상배율(이자비용)"] < 1
        # print(
        #     group.iloc[i]["이자보상배율(이자비용)"],
        #     group.iloc[i + 1]["이자보상배율(이자비용)"],
        #     group.iloc[i + 2]["이자보상배율(이자비용)"],
        # )
        # print("==========")
        cond4 = pd.isnull(group.iloc[i]["이자보상배율(이자비용)"])
        cond5 = pd.isnull(group.iloc[i + 1]["이자보상배율(이자비용)"])
        cond6 = pd.isnull(group.iloc[i + 2]["이자보상배율(이자비용)"])

        if cond1 & cond2 & cond3:
            # print(group["거래소코드"])
            df.loc[group.index[i + 2], "이보배부실"] = 1
        elif cond4 or cond5 or cond6:
            df.loc[group.index[i + 2], "이보배부실"] = None
        else:
            df.loc[group.index[i + 2], "이보배부실"] = 0

        # print(group[i]['이자보상배율(이자비용)'])

In [32]:
## 사용할 이보배부실 회사
bank2 = df.loc[df["이보배부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
bank2_com = bank2[["거래소코드", "회계년도"]]
bank2_com

,거래소코드,회계년도
19,50120,2020
36,217730,2017
48,114190,2018
64,348150,2022
67,35290,2014
...,...,...
8508,322190,2022
8511,266170,2019
8531,149300,2022
8549,343090,2022


In [33]:
nan_list = []

for code, year in zip(bank2_com["거래소코드"], bank2_com["회계년도"]):
    test = df[df["거래소코드"] == code]
    year_index = test[test["회계년도"] == year].index
    if (code != df["거래소코드"].iloc[year_index[0] - 2]) | (year - 2) != df["회계년도"].iloc[
        year_index[0] - 2
    ]:
        nan_list.append(code)

print(nan_list)

[]


In [34]:
# df[df["거래소코드"] == year_nan_list]
for code in nan_list:
    df = df[df["거래소코드"] != code]

In [35]:
year_nan_list = []

for code, group in grouped:
    # print(code, group)
    # print(len(group))
    len_group = len(group)
    if len_group < 3:
        year_nan_list.append(code)
print(year_nan_list)

[18700, 58970, 59270, 67370, 86710, 99390, 99430, 102370, 107600, 112290, 114840, 129920, 136410, 137080, 137310, 146320, 159010, 163730, 168360, 169670, 176590, 187660, 189330, 199800, 200350, 203400, 204610, 236810, 239890, 247660, 248020, 251120, 254160, 257720, 260970, 261780, 262840, 270660, 273640, 276040, 276730, 277810, 288980, 289220, 290090, 291810, 294570, 296640, 303530, 308080, 310210, 311320, 311960, 314140, 314930, 315640, 318660, 321820, 322310, 328130, 330350, 333620, 334970, 338220, 340930, 341170, 346010, 347700, 348030, 348340, 348370, 351330, 352480, 352700, 352910, 353590, 354390, 356890, 357230, 357580, 357880, 360070, 361390, 361570, 361670, 362320, 363250, 363260, 365270, 365340, 365900, 366030, 367000, 368600, 368770, 368970, 371950, 372170, 372800, 373200, 376180, 376930, 376980, 377030, 377220, 377330, 377450, 377480, 377740, 378800, 379390, 380540, 382480, 382800, 382840, 382900, 383310, 383930, 388050, 388610, 388720, 389030, 389140, 389260, 389470, 390110

In [36]:
# df[df["거래소코드"] == year_nan_list]
for code in year_nan_list:
    df = df[df["거래소코드"] != code]

In [37]:
# # 사용할 감사부실 회사
insolvent_com = df.loc[df["이보배부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
insolvent_codes = insolvent_com["거래소코드"].unique()

non_insolvent_com = df[~df["거래소코드"].isin(insolvent_codes)]  # 부실선정 기업 제거
non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

non_insolvent_com.isna().sum()

print(len(insolvent_codes))
print(len(non_insolvent_com) - len(non_insolvent_com.isna().sum()))
print(len(non_insolvent_com.isna().sum()))
print(len(df["거래소코드"].unique()))

507
474
7
988


C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\1599080698.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [38]:
ratio_df = pd.concat([insolvent_com, non_insolvent_com])

##### 현금흐름 부실

In [39]:
flow_df = pd.DataFrame()

In [40]:
df = listed_df.copy()

In [41]:
df["현금흐름부실"] = None

In [42]:
grouped = df.groupby("거래소코드")

In [43]:
for code, group in grouped:
    # print(code, group)
    group_len = len(group)
    # print(group_len)
    for i in range(group_len - 2):
        cond1 = group.iloc[i]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"] < 0
        cond2 = group.iloc[i + 1]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"] < 0
        cond3 = group.iloc[i + 2]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"] < 0
        # print(
        #     group.iloc[i]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"],
        #     group.iloc[i + 1]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"],
        #     group.iloc[i + 2]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"],
        # )
        # print("==========")
        cond4 = pd.isnull(group.iloc[i]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"])
        cond5 = pd.isnull(group.iloc[i + 1]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"])
        cond6 = pd.isnull(group.iloc[i + 2]["[U01410000]영업활동으로 인한 현금흐름(*)(천원)"])

        if cond1 & cond2 & cond3:
            # print(group["거래소코드"])
            df.loc[group.index[i + 2], "현금흐름부실"] = 1
        elif cond4 or cond5 or cond6:
            df.loc[group.index[i + 2], "현금흐름부실"] = None
        else:
            df.loc[group.index[i + 2], "현금흐름부실"] = 0

        # print(group[i]['[U01410000]영업활동으로 인한 현금흐름(*)(천원)'])

In [44]:
# # 사용할 현금흐름부실 회사
bank3 = df.loc[df["현금흐름부실"] == 1].drop_duplicates(subset=["거래소코드"], keep="first")
bank3_com = bank3[["거래소코드", "회계년도"]]
bank3_com

,거래소코드,회계년도
36,217730,2017
48,114190,2018
64,348150,2022
72,35290,2019
96,29480,2020
...,...,...
8493,227420,2018
8508,322190,2022
8529,149300,2020
8549,343090,2022


In [45]:
nan_list = []

for code, year in zip(bank3_com["거래소코드"], bank3_com["회계년도"]):
    test = df[df["거래소코드"] == code]
    year_index = test[test["회계년도"] == year].index
    if (code != df["거래소코드"].iloc[year_index[0] - 2]) | (year - 2) != df["회계년도"].iloc[
        year_index[0] - 2
    ]:
        nan_list.append(code)

print(nan_list)

[299660, 11330]


In [46]:
# df[df["거래소코드"] == year_nan_list]
for code in nan_list:
    df = df[df["거래소코드"] != code]

In [47]:
year_nan_list = []

for code, group in grouped:
    # print(code, group)
    # print(len(group))
    len_group = len(group)
    if len_group < 3:
        year_nan_list.append(code)
print(year_nan_list)

[18700, 58970, 59270, 67370, 86710, 99390, 99430, 102370, 107600, 112290, 114840, 129920, 136410, 137080, 137310, 146320, 159010, 163730, 168360, 169670, 176590, 187660, 189330, 199800, 200350, 203400, 204610, 236810, 239890, 247660, 248020, 251120, 254160, 257720, 260970, 261780, 262840, 270660, 273640, 276040, 276730, 277810, 288980, 289220, 290090, 291810, 294570, 296640, 303530, 308080, 310210, 311320, 311960, 314140, 314930, 315640, 318660, 321820, 322310, 328130, 330350, 333620, 334970, 338220, 340930, 341170, 346010, 347700, 348030, 348340, 348370, 351330, 352480, 352700, 352910, 353590, 354390, 356890, 357230, 357580, 357880, 360070, 361390, 361570, 361670, 362320, 363250, 363260, 365270, 365340, 365900, 366030, 367000, 368600, 368770, 368970, 371950, 372170, 372800, 373200, 376180, 376930, 376980, 377030, 377220, 377330, 377450, 377480, 377740, 378800, 379390, 380540, 382480, 382800, 382840, 382900, 383310, 383930, 388050, 388610, 388720, 389030, 389140, 389260, 389470, 390110

In [48]:
# df[df["거래소코드"] == year_nan_list]
for code in year_nan_list:
    df = df[df["거래소코드"] != code]

In [49]:
# # 사용할 감사부실 회사
insolvent_com = df.loc[df["현금흐름부실"] == 1].drop_duplicates(
    subset=["거래소코드"], keep="first"
)
insolvent_codes = insolvent_com["거래소코드"].unique()

non_insolvent_com = df[~df["거래소코드"].isin(insolvent_codes)]  # 부실선정 기업 제거
non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

non_insolvent_com.isna().sum()

print(len(insolvent_codes))
print(len(non_insolvent_com) - len(non_insolvent_com.isna().sum()))
print(len(non_insolvent_com.isna().sum()))
print(len(df["거래소코드"].unique()))

350
629
7
986


C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\2403248965.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_insolvent_com.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [50]:
flow_df = pd.concat([insolvent_com, non_insolvent_com])

##### 데이터프레임 concat ( code_df, ratio_df, flow_df )

In [51]:
df_list = [code_df, ratio_df, flow_df]
kskqkn = pd.concat(df_list)

In [52]:
kskqkn = kskqkn.sort_values(by=["거래소코드", "회계년도"], ascending=True)

In [53]:
# 상장 부실판단
kskqkn["부실판단"] = None
cond1 = (kskqkn["감사부실"] == 1) | (kskqkn["이보배부실"] == 1) | (kskqkn["현금흐름부실"] == 1)
cond2 = (
    (~cond1) & (kskqkn["감사부실"] == 0) | (kskqkn["이보배부실"] == 0) | (kskqkn["현금흐름부실"] == 0)
)

kskqkn.loc[cond1, "부실판단"] = 1
kskqkn.loc[cond2, "부실판단"] = 0

In [54]:
## 상장기업
# 부실기업 데이터만 출력
bank1 = kskqkn[kskqkn["부실판단"] == 1]
bank1.drop_duplicates(subset="거래소코드", keep="first", inplace=True)

# 정상기업 데이터만 남기기
bank1_code = bank1["거래소코드"].unique()  # 부실로 선정된 기업의 거래소코드
non_bank1 = kskqkn[~kskqkn["거래소코드"].isin(bank1_code)]  # 부실선정 기업 제거
non_bank1.drop_duplicates(subset="거래소코드", keep="last", inplace=True)

# 부실기업과 정상기업 데이터프레임 합치기
final1 = pd.concat([bank1, non_bank1])
final1.reset_index(drop=True, inplace=True)

C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\919437452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bank1.drop_duplicates(subset="거래소코드", keep="first", inplace=True)
C:\Users\jay_hook\AppData\Local\Temp\ipykernel_3576\919437452.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_bank1.drop_duplicates(subset="거래소코드", keep="last", inplace=True)


In [55]:
final1 = final1[["회사명", "거래소코드", "회계년도", "부실판단"]]

In [56]:
final1.to_csv("../datasets/Labeling/상장_부실_Labeling.csv")